In [ ]:
import pandas as pd
import numpy as np
import collections
#set the number of columns and rows of the dataframe to display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

In [ ]:
df = pd.read_csv("D:\\order_brush_order.csv")
df


,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31


In [ ]:
def processdate(row):
    #split space(date[0] & time[1]),then split hyphen(year[0],month[1],day[2]) **split function will split str into list
    day = int(row['event_time'].split(' ')[0].split('-')[2]) 
    #similar to above
    hour = int(row['event_time'].split(' ')[1].split(':')[0])
    minute = int(row['event_time'].split(' ')[1].split(':')[1])
    sec = int(row['event_time'].split(' ')[1].split(':')[2])
    #as there is only day 27&28, then set day 27 first second as standard, change time unit to seconds
    return ((day-27)*86400 + hour*3600 + minute*60 + sec)

In [ ]:
#axis=1 means apply to each row of 'event_time', perform the func and insert the result in a new column 'time', dunno how to replace lambda with normal function 
df['time'] = df.apply(lambda row: processdate(row),axis=1)
#drop columns orderid and event_time, inplace=True means do the modification on the same dataframe instead of returning a new dataframe
df.drop(columns = ['event_time'], inplace = True)

#sort shopid(first) and time(second) in df
df.sort_values(by=['shopid','time'], inplace = True)
#reset index after sorting, drop=True to prevent old index to be added as new column
df.reset_index(drop=True, inplace = True)


In [ ]:
#construct dataframe named answer, the dataframe index is sorted shopid (create unique shopid using pandas.unique)
answer = pd.DataFrame(index = np.sort(df['shopid'].unique()))
#create new column named userid with values=0
answer['userid'] = '0'
#create array of dataframe df (easier to manipulate)
arr = np.array(df)
#loop through each order instance
for i in range(len(arr)):
    shopid=arr[i][1] #save current shopid
    #if(shopid!=147941492):
    #    continue
    starttime=arr[i][3] #save currrent time
    index=i  #save current index
    orderid_temp=[arr[i][0]]  #create a list of orderid and insert current orderid in it(renewed when starttime change)  
    orderid_userid_templist=[]
    buyer_unique={arr[i][2]}
    if(i==0 or shopid!=arr[i-1][1]):
        orderid_userid_list=[]
    m=0
    while True:
        #count=0    #variable to count number of the buyer mode
        index+=1
        #break when reach end of array or span time exceeded 3600secs or shopid change
        if(index==len(arr) or arr[index][3]-starttime>3600 or arr[index][1]!=shopid):
            break
        orderid_temp.append(arr[index][0]) #add orderid of all looped instance
        buyer_unique.update([arr[index][2]]) #add buyerid of all looped instance in set     
        orderid_userid_templist.append([str(arr[index][0]),str(arr[index][2])])
        #if concentration>=3.0, find mode and number of occurences of mode in buyer list
        if (float(len(orderid_temp))/float(len(buyer_unique))>=3.0):
            orderid_userid_list.extend(orderid_userid_templist)

    #update final answer when order instance reach end of array or next shopid is different
    if(i==len(arr)-1 or arr[i+1][1]!=shopid):
        #print(shopid)
        maxNum=0
        new_orderid_userid_list=[]
        for elem in orderid_userid_list:
            if(elem not in new_orderid_userid_list):
                new_orderid_userid_list.append(elem)
        new_orderid_userid_list.sort(key=lambda x:x[1])
        #print(new_orderid_userid_list) 
        
        temp=0
        for num in range(len(new_orderid_userid_list)):
            count=1
            if(new_orderid_userid_list[num][1]==temp):
                continue
            temp=new_orderid_userid_list[num][1]
            for num1 in range((num+1), len(new_orderid_userid_list)):
                if(new_orderid_userid_list[num1][1]==temp):
                        count+=1
            if(count>maxNum):
                maxNum=count
       # print(maxNum)
        switch=0
        tempBuyerid=0
        for num in range(len(new_orderid_userid_list)):
            if(new_orderid_userid_list[num][1]==tempBuyerid):
                continue
            tempBuyerid=new_orderid_userid_list[num][1]
            temp1=sum(x.count(tempBuyerid) for x in new_orderid_userid_list)
            if(temp1==maxNum):
                #print(tempBuyerid)
                if(switch==1):
                    answer.loc[shopid]['userid']+='&'
                    answer.loc[shopid]['userid']+=str(tempBuyerid)
                else:
                    answer.loc[shopid]['userid']=str(tempBuyerid)
                    switch=1
        


In [ ]:
submit = pd.DataFrame(data = {'shopid':answer.index,'userid':answer['userid']})
#index=row name, header=column name, r'str' means raw string
submit.to_csv (r'final9.csv', index = False, header=True)
